# **In Class Assignment: Sentiment Analysis**

## Name: KEY
## *IS 5150*

In this in-class assignment we will examine the sentiment of movie reviews using both unsupervised lexicon-based modeling and through supervised classification. We will then leverage tools to interpret the decision of our sentiment analysis model to determine the words and topics associated with positive and negative sentiments.

We begin, as always, be importing our dependencies:

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sklearn
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.decomposition import LatentDirichletAllocation

!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn


np.set_printoptions(precision=2, linewidth=80)

import warnings
warnings.filterwarnings("ignore")

## **Load and Normalize the Dataset**

In [34]:
df = pd.read_csv('/content/movie_reviews.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [43]:
reviews = np.array(df['review'])
sentiments = np.array(df['sentiment'])

test_reviews = reviews[45000:]
test_sentiments = sentiments[45000:]
sample_review_ids = [2626, 3533, 4012]

### Normalization Functions

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
### THIS IS AN ADDITION
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('but')
stop_words.remove('not')
#####
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

from pprint import pprint
import numpy as np
import re
from bs4 import BeautifulSoup

import spacy
nlp = spacy.load('en_core_web_sm')                                                                                            # dependencies

import unicodedata

!pip install contractions
import contractions

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]                                                                         # html parsing
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def tokenize_text(text):                                                                                                      # text tokenization
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences] 
    return word_tokens

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')                            # accent removal
    return text

def expand_contractions(text):                                                                                                # expand contractions
    expanded_words = []
    for word in text.split():
        expanded_words.append(contractions.fix(word))
        expanded_text = ' '.join(expanded_words)
        return expanded_text

def remove_special_characters(text, remove_digits=False):                                                                    # special character removal
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def simple_stemmer(text):                                                                                                   # stemmer
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)                                                                                                        # lemmatizer
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def remove_stopwords(text, is_lower_case=False, stopwords=stop_words):                                                   # stopword removal
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

## **Sentiment Analysis with VADER**
### Build the Model

In [46]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    review_text = strip_html_tags(review)
    review_text = remove_accented_chars(review_text)
    review_text = expand_contractions(review_text)

    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review_text)

    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative', 'Neutral']], 
                                                              codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment


### Predict Sentiment of Sample Reviews

In [49]:
for reviews, sentiments in zip(reviews[sample_review_ids], sentiments[sample_review_ids]):
    print('REVIEW:', reviews)
    print('Actual Sentiment:', sentiments)
    pred = analyze_sentiment_vader_lexicon(reviews, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: I'm a fan of Zhang Yimou and finally found this DVD title from the shelves of a Shenzhen bookstore after a long search at many places.<br /><br />This is a huge departure from previous Zhang Yimou work, esp in terms of style and locale. The director himself has said that this is the first and only time he'll ever attempt to make a black comedy set in contemporary China. You may even say this work is experimental in nature, compared to his other well known big budget and formal pieces.<br /><br />Filmed with a hand-held camera and wide angle lens throughout the duration of the whole film, the quick pace editing and high energy performance & naturalistic tone never let you go once it grips you from the start. It presents a very realistic account of modern Chinese urban sensibilities, which in this case is set in Beijing. If you appreciate and love this kind of black humor, you will love this film totally. Also look out for hilarious cameos by Zhao Benshan (Happy Times)and the dir

**What impact does the threshold value have on the classification of sentiments within reviews?**

**Do you agree with all of the ratings? If not, what words or phrases might be throwing off the sentiment score?**

## **Next let's predict sentiment via machine learning, using several classification algorithms...**

### Respeficy the dataset, then build a training and test set

In [69]:
reviews = np.array(df['review'])
sentiments = np.array(df['sentiment'])

In [70]:
train_reviews = reviews[:350]
train_sentiments = sentiments[:350]
test_reviews = reviews[350:450]
test_sentiments = sentiments[350:450]

In [71]:
train_reviews[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

### Normalize the corpora Function

In [65]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=False,                                                # define normalize corpus function
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
           doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [72]:
norm_train_reviews = normalize_corpus(train_reviews)
norm_test_reviews = normalize_corpus(test_reviews)

In [74]:
norm_train_reviews[0]

'one reviewer mention watch oz episode hook right exactly happen first thing strike I oz brutality unflinche scene violence set right word go trust I not show faint hearted timid show pull no punch regard drug sex violence hardcore classic use word call oz nickname give oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inward privacy not high agenda em city home many aryan muslims gangsta latinos christians italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show would not dare forget pretty picture paint mainstream audience forget charm forget romance oz not mess around first episode I ever see strike I nasty surreal I could not say I ready but I watch I develop taste oz get accustomed high level graphic violence not violence but injustice crooked guard sell nickel inmate kill order get away well mannered middle class inmate turn prison bitch due lack s

### Feature Engineering

In [75]:
# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

In [76]:
# transform test reviews into features
tv_test_features = tv.transform(norm_test_reviews)

In [77]:
print(tv_train_features.shape)
print(tv_test_features.shape)

(350, 45535)
(100, 45535)


### Model Training, Prediction, and Performance Eval

In [78]:
lr = LogisticRegression(penalty='l2', max_iter=100, C=1)
svm = SGDClassifier(loss='hinge', max_iter=100)

#### Logistic Regression with TF-IDF Features

In [79]:
def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions  

lr_tfidf_predictions = train_predict_model(classifier=lr, 
                                               train_features=tv_train_features, train_labels=train_sentiments,
                                               test_features=tv_test_features, test_labels=test_sentiments)


In [80]:
print(classification_report(test_sentiments, lr_tfidf_predictions))

              precision    recall  f1-score   support

    negative       0.65      1.00      0.79        53
    positive       1.00      0.40      0.58        47

    accuracy                           0.72       100
   macro avg       0.83      0.70      0.68       100
weighted avg       0.82      0.72      0.69       100



**Which metric would be most appropriate to use in this case and why?**

Accuracy isn't a bad metric in this case since the class distribution isn't too unequal. If we're saying neither false positive or negatives matter more, then probably accuracy or f-1 score would suffice.

## **Interpretation Time!**

First let's examine predicted probabilities to see how confident our model was in its predictions...

In [81]:
model = lr.fit(tv_train_features, train_sentiments)

In [82]:
predicted_probas = list(zip(*model.predict_proba(tv_test_features)))[0]

In [83]:
predictions_df = pd.DataFrame()
predictions_df['Reviews'] = test_reviews
predictions_df['True Sentiment'] = test_sentiments
predictions_df['Predicted Sentiment'] = lr_tfidf_predictions
predictions_df['Probability_Negative'] = predicted_probas

In [84]:
predictions_df

,Reviews,True Sentiment,Predicted Sentiment,Probability_Negative
0,Man with the Screaming Brain certainly isn't a...,positive,negative,0.546701
1,You should never ever even consider to watch t...,negative,negative,0.584740
2,Tell the truth I’m a bit stun to see all these...,negative,negative,0.615391
3,This film held my interest enough to watch it ...,positive,negative,0.527715
4,"Like the previous poster, I am from northern V...",negative,negative,0.574476
...,...,...,...,...
95,This movie must be in line for the most boring...,negative,negative,0.644545
96,"Down at the Movie Gallery, I saw a flick I jus...",negative,negative,0.630940
97,"i am finally seeing the El Padrino movie, from...",positive,positive,0.481828
98,I thrive on cinema....but there is a limit. A ...,negative,negative,0.552616


**Sort the predictions_df by predicted probability to find movies with high, moderate and low probabilities to examine a) whether the predictions were correct and b) try to identify what sorts of words or phrases led to high or low probabilities of a negative sentiment**

## Topic Modeling for Positive \& Negative Sentiments

In [85]:
norm_reviews = norm_train_reviews + norm_test_reviews

### Display Topics for **Positive** Reviews

In [95]:
# get tf-idf features for only positive reviews
positive_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'positive']
ptvf = CountVectorizer(min_df = 0.02, max_df=0.75)
ptvf_features = ptvf.fit_transform(positive_reviews)

In [96]:
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(ptvf_features)

LatentDirichletAllocation(random_state=0)

In [97]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, ptvf_features, ptvf)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.051533  0.005436       1        1  19.849490
9      0.010293 -0.002872       2        1  18.301360
2     -0.026114  0.001370       3        1  12.319473
1     -0.077733  0.083070       4        1  10.765145
0     -0.067015  0.064579       5        1  10.740055
3     -0.017151  0.043878       6        1  10.109437
6      0.173758  0.086305       7        1   7.092438
8     -0.005665 -0.077302       8        1   4.675603
4     -0.024354 -0.126819       9        1   3.190232
5      0.085513 -0.077644      10        1   2.956767, topic_info=       Term        Freq       Total Category  logprob  loglift
547   movie  381.000000  381.000000  Default  30.0000  30.0000
324    film  402.000000  402.000000  Default  29.0000  29.0000
113     but  322.000000  322.000000  Default  28.0000  28.0000
621    play   82.000000   82.000000  Default  27.0000  27.0000
946   world   68.000000   68.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
78   become    2.623466   38.824866  Topic10  -5.2928   0.8265
792   start    2.623454   32.196863  Topic10  -5.2928   1.0137
830    take    2.623454   91.388483  Topic10  -5.2928  -0.0296
885     try    2.623453   33.185085  Topic10  -5.2928   0.9835
160  comedy    2.623451   40.648728  Topic10  -5.2928   0.7806

[651 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
4         4  0.134972  accent
4         5  0.269945  accent
4         6  0.269945  accent
4         9  0.269945  accent
8         1  0.278444     act
...     ...       ...     ...
956       4  0.052551   young
956       5  0.070068   young
956       6  0.035034   young
956       7  0.175171   young
956       8  0.087586   young

[2062 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 3, 2, 1, 4, 7, 9, 5, 6])

### Display Topics for **Negative** Reviews

In [98]:
# get tf-idf features for only negative reviews
negative_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'negative']
ntvf = CountVectorizer(min_df = 0.02, max_df=0.75)
ntvf_features = ntvf.fit_transform(negative_reviews)

In [99]:
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(ntvf_features)

LatentDirichletAllocation(random_state=0)

In [100]:
pyLDAvis.sklearn.prepare(lda_tf, ntvf_features, ntvf)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.012832  0.003218       1        1  26.430750
2     -0.016248 -0.029016       2        1  11.380349
8     -0.010439 -0.067213       3        1  11.345384
9      0.035962 -0.032614       4        1  11.139466
1      0.009325 -0.014028       5        1  10.312952
5     -0.015195  0.004165       6        1   8.640397
3      0.045116  0.012382       7        1   8.232523
4      0.008243  0.165437       8        1   5.876625
0      0.095476 -0.033611       9        1   5.043467
6     -0.165072 -0.008719      10        1   1.598088, topic_info=      Term        Freq       Total Category  logprob  loglift
533  movie  529.000000  529.000000  Default  30.0000  30.0000
309   film  408.000000  408.000000  Default  29.0000  29.0000
338  funny   53.000000   53.000000  Default  28.0000  28.0000
63     bad  202.000000  202.000000  Default  27.0000  27.0000
361   good  151.000000  151.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
107    but    3.686847  386.505620  Topic10  -4.4200  -0.5160
359     go    2.762668  148.975993  Topic10  -4.7085   0.1488
494   make    2.759656  223.714068  Topic10  -4.7096  -0.2589
383   help    1.677886   32.186502  Topic10  -5.2072   1.1823
830  thing    1.677884   88.316616  Topic10  -5.2072   0.1730

[675 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.526474  ability
0         9  0.394855  ability
3         1  0.162121   accent
3         3  0.324242   accent
3         5  0.081060   accent
...     ...       ...      ...
933       6  0.299200     zero
934       1  0.109718   zombie
934       4  0.219435   zombie
934       5  0.073145   zombie
934       8  0.621733   zombie

[2115 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 9, 10, 2, 6, 4, 5, 1, 7])

**Examine the 30 most salient terms as well as the topics amongst the positive and negative reviews, are there any interesting differences in the most salient terms or topics between positive and negative movie reviews?**